<a href="https://colab.research.google.com/github/kinoshita-hiroki/kaggle/blob/main/Titanic/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# todo
- 連続値

# 準備
KaggleのタイタニックコンペティションのページからDataをダウンロードする → [Download All]

https://www.kaggle.com/c/titanic/data#

自分のGoogle Driveを開いて任意の場所にタイタニックデータ3つ(gender_submission.csv, test.csv, train.csv)をアップロードする
※ここでは マイドライブ/KaggleLearning/Titanic ディレクトリ配下に置いた

ランタイム接続し、左側メニューの「ファイル」を選択して出てきたウィンドウの中の「ドライブをマウント」を押下すと以下のコマンドが出てくる。これを実行する。

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [36]:
import numpy as np

## ニューラルネット

In [4]:
from keras.layers import Dense, Dropout
from keras.models import Sequential
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [7]:
df_tr = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/kaggle/Titanic/train.csv")
df_te = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/kaggle/Titanic/test.csv")

In [8]:
df_tr.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [20]:
df_tr.loc[:, "Embarked"].value_counts(dropna=False)

S      644
C      168
Q       77
NaN      2
Name: Embarked, dtype: int64

# データ解析
メモ
- 年齢とFareは連続値
- チケット番号が同じものは家族だから生存率高そうだけど、面倒だから特徴から外してもいいかも
- Parch, SibSp両方独身かそれ以外で見た方がいいかも
- すべての男は消耗品である。
- テストデータに同じキャビンがあればリークしないのかな？
- Nanが存在する->Age, Cabin, Embarked
- Cabin != nullのが生存率高い


In [22]:
import pandas as pd
df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/kaggle/Titanic/train.csv")

In [23]:
df_train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [26]:
df_train.loc[:, "Cabin"].value_counts(dropna=False)

NaN            687
B96 B98          4
C23 C25 C27      4
G6               4
C22 C26          3
              ... 
E17              1
A24              1
C85              1
B39              1
E12              1
Name: Cabin, Length: 148, dtype: int64

In [30]:
ca_suv = df_train.loc[:, ["Cabin", "Survived"]]

In [45]:
ca_suv[ca_suv["Cabin"].isnull()].loc[:, "Survived"].value_counts()

0    481
1    206
Name: Survived, dtype: int64

In [47]:
ca_suv.loc[:, "Survived"].value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [38]:
df_train.loc[:, "Embarked"].value_counts(dropna=False)

S      644
C      168
Q       77
NaN      2
Name: Embarked, dtype: int64

In [ ]:
pd.cut(df_train["Age"], 5)

0      (16.336, 32.252]
1      (32.252, 48.168]
2      (16.336, 32.252]
3      (32.252, 48.168]
4      (32.252, 48.168]
             ...       
886    (16.336, 32.252]
887    (16.336, 32.252]
888                 NaN
889    (16.336, 32.252]
890    (16.336, 32.252]
Name: Age, Length: 891, dtype: category
Categories (5, interval[float64]): [(0.34, 16.336] < (16.336, 32.252] < (32.252, 48.168] <
                                    (48.168, 64.084] < (64.084, 80.0]]

In [ ]:
df_train.pivot_table(values="Survived", index="Embarked")

,Survived
Embarked,
C,0.553571
Q,0.389610
S,0.336957


In [ ]:
df_train.pivot_table(values="Survived", index="SibSp", aggfunc=sum)

,Survived
SibSp,
0,210
1,112
2,13
3,4
4,3
5,0
8,0


In [ ]:
pd.cut()

TypeError: ignored

# モデル

In [ ]:
import numpy as np

def round(pred):
  return np.round(pred)

In [ ]:
import xgboost as xgb
from sklearn.metrics import log_loss

dtr = xgb.DMatrix("/content/drive/MyDrive/Colab Notebooks/kaggle/Titanic/train.csv?format=csv&label_column=1")
dte = xgb.DMatrix("/content/drive/MyDrive/Colab Notebooks/kaggle/Titanic/test.csv?format=csv")

params = {'objective': 'binary:logistic', 'slient': 1, 'random_state': 71}

model = xgb.train(params, dtr, 50)

va_pred = model.predict(dte)

print(round(va_pred))
print(len(va_pred))

In [ ]:
import lightgbm as lgb
from sklearn.metrics import log_loss
import pandas as pd
from sklearn.model_selection import train_test_split

categorical_features = ['Sex', 'Ticket', 'Cabin', 'Embarked']

ptr = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/kaggle/Titanic/train.csv")
pte = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/kaggle/Titanic/test.csv")

ptr["TrainFlag"] = True
pte["TrainFlag"] = False

df_all = ptr.append(pte)
df_all.index = df_all["PassengerId"]

df_all.drop(["PassengerId", "Name"], axis = 1, inplace = True)

df_all = pd.get_dummies(df_all, drop_first=True)

df_train = df_all[df_all["TrainFlag"] == True]
df_train = df_train.drop(["TrainFlag"], axis = 1)

df_test = df_all[df_all["TrainFlag"] == False]
df_test = df_test.drop(["TrainFlag"], axis = 1)
df_test = df_test.drop(["Survived"], axis = 1)

y_train = df_train["Survived"].values
X_train = df_train.drop("Survived", axis=1).values

lgb_train = lgb.Dataset(X_train, y_train)

params = {'objective': 'binary', 'seed': 71, 'verbose': 0, 'metrics': 'binary_logloss'}

model = lgb.train(params, lgb_train, num_boost_round=100)

va_pred = model.predict(df_test)

print(round(va_pred))

[0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1.
 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0.
 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0.
 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1.
 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0.
 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1.
 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0.

In [ ]:
[int(s) for s in round(va_pred)]

[0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,


In [ ]:
pte.loc[:, "PassengerId"]

0       892
1       893
2       894
3       895
4       896
       ... 
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64

In [ ]:
pd.DataFrame({
    "PassengerId": pte.loc[:, "PassengerId"],
    "Survived": [int(s) for s in round(va_pred)]
}).set_index("PassengerId").to_csv("/content/drive/MyDrive/Colab Notebooks/kaggle/Titanic/result.csv")

In [ ]:
pte

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,TrainFlag
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,False
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,False
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,False
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,False
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,False
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,False
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,False
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,False
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,False
